### Step 1: Setup Environment and Load Datasets

1. **Import Libraries**: We'll import PyTorch, torchvision, and other necessary libraries.
2. **Load Datasets**: Load MNIST and CIFAR10 datasets using torchvision.
3. **Data Loaders**: Create data loaders for both datasets for easy batch processing.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Device configuration - GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# MNIST dataset
mnist_train = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
mnist_test = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())
mnist_train_loader = DataLoader(dataset=mnist_train, batch_size=64, shuffle=True)
mnist_test_loader = DataLoader(dataset=mnist_test, batch_size=64, shuffle=False)

# CIFAR10 dataset
cifar_train = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transforms.ToTensor(), download=True)
cifar_test = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transforms.ToTensor())
cifar_train_loader = DataLoader(dataset=cifar_train, batch_size=64, shuffle=True)
cifar_test_loader = DataLoader(dataset=cifar_test, batch_size=64, shuffle=False)


### Step 2: Model Definition

We'll define simple CNN architectures suitable for each dataset. MNIST images are grayscale and smaller, while CIFAR10 images are color and larger.

#### MNIST CNN Model:
- Simple architecture with a couple of convolutional layers.

#### CIFAR10 CNN Model:
- A bit more complex due to the nature of the dataset (color images).

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# CNN for MNIST
class MNIST_CNN(nn.Module):
    def __init__(self):
        super(MNIST_CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(7*7*64, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(x.size(0), -1) # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# CNN for CIFAR10
class CIFAR10_CNN(nn.Module):
    def __init__(self):
        super(CIFAR10_CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(8*8*128, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(x.size(0), -1) # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


Awesome! Now let's implement the FGSM and PGD functions. 😊💻

### Step 3: FGSM Function
The FGSM method creates adversarial examples by adding a small perturbation to the original image in the direction of the gradient of the loss with respect to the input image.

### Step 4: PGD Function
PGD is a more powerful attack compared to FGSM. It applies the perturbation iteratively and projects the perturbed image back into the allowed range after each step.

In [ ]:
def fgsm_attack(image, epsilon, data_grad):
    """
    Apply FGSM attack.
    :param image: Original image
    :param epsilon: Perturbation amount
    :param data_grad: Gradient of the loss w.r.t. input image
    :return: Perturbed image
    """
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon * sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

def pgd_attack(model, image, label, epsilon, alpha, iters):
    """
    Apply PGD attack.
    :param model: The model to fool
    :param image: Original image
    :param label: True label of the image
    :param epsilon: Perturbation amount
    :param alpha: Step size
    :param iters: Number of iterations
    :return: Perturbed image
    """
    # Initialize perturbation
    perturbation = torch.zeros_like(image).to(device)
    perturbation.requires_grad = True

    for _ in range(iters):
        outputs = model(image + perturbation)
        loss = F.cross_entropy(outputs, label)
        model.zero_grad()
        loss.backward()
        with torch.no_grad():
            # Update perturbation
            perturbation += alpha * perturbation.grad.sign()
            perturbation = torch.clamp(perturbation, -epsilon, epsilon)
    return image + perturbation


Great! Let's proceed to the testing part where we'll apply FGSM and PGD attacks on some sample images from the MNIST and CIFAR10 datasets. 😊🔬

### Step 5: Testing FGSM and PGD

We'll:
1. Load pretrained models or train simple models for MNIST and CIFAR10.
2. Select a few sample images from both datasets.
3. Apply FGSM and PGD attacks on these samples.
4. Visualize the results to see the effect of the attacks.

In [ ]:
import matplotlib.pyplot as plt

def test(model, device, test_loader, attack, epsilon):
    """
    Test the model on attacked images.
    :param model: Trained model
    :param device: Device to use
    :param test_loader: DataLoader for test data
    :param attack: Attack function (FGSM or PGD)
    :param epsilon: Epsilon for FGSM, (epsilon, alpha, iters) for PGD
    """
    # Set model to evaluation mode
    model.eval()
    correct = 0
    adv_examples = []

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        data.requires_grad = True

        # Forward pass
        output = model(data)
        init_pred = output.max(1, keepdim=True)[1]

        # If the initial prediction is wrong, don't bother attacking, just move on
        if init_pred.item() != target.item():
            continue

        # Calculate the loss
        loss = F.cross_entropy(output, target)

        # Zero all existing gradients
        model.zero_grad()

        # Backward pass
        loss.backward()
        data_grad = data.grad.data

        # Call attack
        if attack == fgsm_attack:
            perturbed_data = attack(data, epsilon, data_grad)
        else:  # PGD attack
            perturbed_data = attack(model, data, target, *epsilon)

        # Re-classify the perturbed image
        output = model(perturbed_data)

        # Check for success
        final_pred = output.max(1, keepdim=True)[1]
        if final_pred.item() == target.item():
            correct += 1
        else:
            # Save some adv examples for visualization later
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append((init_pred.item(), final_pred.item(), adv_ex))

    # Calculate final accuracy
    final_acc = correct / float(len(test_loader))
    print(f"Test Accuracy = {final_acc} under {attack.__name__} Attack with epsilon={epsilon}")

    # Return the accuracy and an adversarial example
    return final_acc, adv_examples

# Example of testing
model = MNIST_CNN().to(device)
# Load your trained model or train here

# Test FGSM Attack
epsilons = [0.05, 0.1, 0.15]
for eps in epsilons:
    acc, ex = test(model, device, mnist_test_loader, fgsm_attack, eps)
    # Visualize the results

# Test PGD Attack
eps, alpha, iters = 0.2, 0.01, 40
acc, ex = test(model, device, mnist_test_loader, pgd_attack, (eps, alpha, iters))
# Visualize the results
